In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from nlp_chat_bot.model.minilm import MiniLM
from nlp_chat_bot.rag.classic_rag import ClassicRAG
from nlp_chat_bot.vector_store.chroma_vector_store_builder import ChromaVectorStoreBuilder

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
dataset_path = "../data"
vector_store_path = "../chromadb"
model_download_path = "../models"

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=50,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)

embedding_function = MiniLM(model_download_path=model_download_path)
llm_gemini = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
document_loader = None
vector_store = ChromaVectorStoreBuilder(dataset_path,
                                        embedding_function,
                                        vector_store_path,
                                        splitter).build(False)
rag = ClassicRAG(vector_store, llm_gemini)

print("LENGTH", rag.get_num_docs())
docs_retrieved = rag.retrieve(state = {"question": "What is my conclusion in my project report on image inpainting?", "context": []})

print("Num docs:", len(docs_retrieved["context"]))

for i in range(len(docs_retrieved["context"])):
    doc = docs_retrieved["context"][i]
    print("\n\n", "#"*30,"\n")
    print(f"doc {i}: (score: {doc.metadata['score']})")
    print(doc.page_content)

d:\Programs\Anaconda\envs\nlp_project_chatbot\lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


LENGTH 1589
Num docs: 3


 ############################## 

doc 0: (score: 0.903975248336792)
Image Inpainting with Basic Convolutional Networks
Robin Meneust, Ethan Pinto
December 2024
1 Introduction
In the context of our ”AI-Based Image Processing”
course, we worked on this project, in which we repro-
duced and tested a specific image inpainting approach,
defined by the paper ”Context Encoders: Feature Learn-
ing by Inpainting”(Pathak et al., 2016)[1].
Image inpainting consists of filling hole(s) in an im-
age. There exist different methods to do so (e.g. they
compared their results with Photoshop). In this paper,
they used a context encoder trained in an adversarial
way. Basically there is a generator, this is our context
encoder (here an encoder and a decoder) that given an
image of size 128x128 with a dropout region (a ”hole”,
with values set to 0) tries to predict what should be inside
the hole. We focused on the simplest case here for the
dropout region: a square in the center o

In [5]:
rag.invoke(query={"question":"What is my conclusion in my project report on image inpainting?"})["answer"]

'The provided text describes the implementation and experimental results of an image inpainting project using a context encoder.  The experiments showed that increasing the adversarial loss weight improved results, achieving results similar to the original paper, although not perfect.  Some errors persisted, especially on difficult images, regardless of the experiment.\n'